In [ ]:
# >>>>>>>>> Generate training data <<<<<<<<<<<<
from Data import *
import os 

init_conditions = ['topHat'] # This could include other initial conditions as defined in IntialConditions.py
num_samples = [10] # Number of samples for each initial condition, 100 was used in the paper: https://arxiv.org/pdf/2408.05193
wavespeeds = np.arange(1, 5.5, 0.5) # Wavespeeds for the initial conditions
num_ic = len(init_conditions)
NUM_SAMPLES = sum(num_samples) * len(wavespeeds)
N = 128
n_levels = int(np.log2(N))
pts_per_cell = 4
xR = 5
xL = -5
h = (xR-xL)/N

training_label = 'training_data/'
if not os.path.exists(training_label):
    os.makedirs(training_label)

data = np.zeros((NUM_SAMPLES, 1, N*pts_per_cell))
exact = np.zeros((NUM_SAMPLES, 1, N*pts_per_cell))
modes = np.zeros((NUM_SAMPLES, N+2), dtype=object)

count = 0
for i in range(num_ic): 
    for j in range(len(wavespeeds)):
        initial_condition=init_conditions[i]
        a = wavespeeds[j]
        print("Generating data for: ", initial_condition)
        data_train, exact_train, evaluation_pts, modes_train, grid, grid_spacing = generateData(num_samples[i], wavespeed=a, domain=[xL, xR], grid_level_lo=n_levels, grid_level_hi=n_levels, uniformgrid_ratio=1, nonsmooth_ratio=1, data_type='quadrature', ic=initial_condition, save_modal=True, show_plot=False)
        data[count:count+num_samples[i], :, :] = data_train
        exact[count:count+num_samples[i], :, :] = exact_train
        modes[count:count+num_samples[i]] = np.array(modes_train, dtype=object)
        count += num_samples[i]

        # Save the raw data for each parameter setting
        np.savez(training_label + initial_condition + '_N'+str(N) + '_a'+str(a) +'_'+ str(num_samples[i])+'samples.npz', modes=modes_train, grid=grid, grid_spacing=grid_spacing, data=data_train, exact=exact_train)


In [ ]:
# Apply SIAC filter 
from SIACFilter import *
from DiscontinuityDetection import *

# ----- SIAC Parameters
smoothness = -1
siac_kernel_size = '0'

# ----- Apply SIAC filter 
filtered, modes_siac, multiwavelets_siac = apply_siac_filter(modes, smoothness, filter_size=siac_kernel_size, N=N, pts_per_cell=pts_per_cell, xL=xL, xR=xR)
multiwavelets = compute_data_multiwavelets(modes, N=N)

# ----- Save data
np.savez(training_label+str(data.shape[0])+'_samples'+'_siac'+siac_kernel_size, data=data, exact=exact, modes=modes, filtered=filtered, multiwavelets=multiwavelets, multiwavelets_siac=multiwavelets_siac, modes_siac=modes_siac)

In [ ]:
# Generate discontinuity data
from DiscontinuityDetection import *
from Data import *
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
sns.set(rc={'figure.figsize':(6,4)})
sns.color_palette('bright')
sns.set_style("white")
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_style("ticks", rc=custom_params)
sns.set_context("talk")

half_window = 4
threshold = 0.4 

flat_val_threshold=0.01  
max_val_threshold=0.1
disc_type = 'all' 

# Identify raw and SIAC filtered discontinuities in the data using TCD
eval_pts = evaluation_pts
disc_data_siac, exact_disc_data_siac = center_tc_data(filtered, multiwavelets_siac, exact[:, 0, :], eval_pts, half_window=half_window, pts_per_cell=pts_per_cell, 
                                                      threshold=threshold, debug=True)
disc_data_siac_norm, exact_disc_data_siac_norm, min_vals_synthetic, max_vals_synthetic, samples_norm_synthetic = normalize_clean_disc_data(disc_data_siac, exact_disc_data_siac, pts_per_cell=pts_per_cell, 
                                                                                                                                           flat_val_threshold=flat_val_threshold, max_val_threshold=max_val_threshold,
                                                                                                                                           disc_type=disc_type)

In [ ]:
# Load CFD data with exact solution and Identify discontinuities -- DENSITY ONLY

half_window = 4
threshold = 0.05
flat_val_threshold=0.01
max_val_threshold=0.1

# LAX DATA 
# -- Test data: 84 samples from T=1 to T=1.3
file_lax = np.load('cfd_data/Lax/CFD_Lax_time1-1.3_N128_Samples84.npz', allow_pickle=True)
data_lax = file_lax['raw_density']
exact_lax = file_lax['exact_density']
modes_lax = file_lax['raw_density_modes']
times_lax = file_lax['times']

filtered_lax, modes_lax_filtered, multiwavelets_siac_lax = apply_siac_filter(modes_lax, smoothness, modes_type='cfd', filter_size=siac_kernel_size, ghost_cells=10, N=N, pts_per_cell=pts_per_cell, xL=xL, xR=xR)
multiwavelets_siac_lax = compute_data_multiwavelets(modes_lax_filtered, modes_type='cfd', N=N)
disc_data_siac_lax, exact_disc_data_siac_lax = center_tc_data(filtered_lax, multiwavelets_siac_lax, exact_lax, eval_pts, half_window=half_window, pts_per_cell=pts_per_cell, threshold=threshold, debug=True)
disc_data_siac_lax_norm, exact_disc_data_siac_lax_norm, min_vals_lax, max_vals_lax, samples_norm_lax = normalize_clean_disc_data(disc_data_siac_lax, exact_disc_data_siac_lax, pts_per_cell=pts_per_cell, flat_val_threshold=flat_val_threshold, max_val_threshold=max_val_threshold)

# -- Validation data: 85 samples from T=0.7 to 1
file_lax_val = np.load('cfd_data/Lax/CFD_Lax_time0.7-1_N128_Samples85.npz', allow_pickle=True)
data_lax_val = file_lax_val['raw_density']
exact_lax_val = file_lax_val['exact_density']
modes_lax_val = file_lax_val['raw_density_modes']
times_lax_val = file_lax_val['times']

filtered_lax_val, modes_lax_val_filtered, multiwavelets_siac_lax_val = apply_siac_filter(modes_lax_val, smoothness, modes_type='cfd', filter_size=siac_kernel_size, ghost_cells=10, N=N, pts_per_cell=pts_per_cell, xL=xL, xR=xR)
multiwavelets_siac_lax_val = compute_data_multiwavelets(modes_lax_val_filtered, modes_type='cfd', N=N)
disc_data_siac_lax_val, exact_disc_data_siac_lax_val = center_tc_data(filtered_lax_val, multiwavelets_siac_lax_val, exact_lax_val, eval_pts, half_window=half_window, pts_per_cell=pts_per_cell, threshold=threshold, debug=True)
disc_data_siac_lax_val_norm, exact_disc_data_siac_lax_val_norm, min_vals_lax_val, max_vals_lax_val, samples_norm_lax_val = normalize_clean_disc_data(disc_data_siac_lax_val, exact_disc_data_siac_lax_val, pts_per_cell=pts_per_cell, flat_val_threshold=flat_val_threshold, max_val_threshold=max_val_threshold)


# SOD DATA
# -- Test data: 65 samples from T=1.5 to T=2
file_sod = np.load('cfd_data/Sod/CFD_Sod_time1.5-2_N128_Samples65.npz', allow_pickle=True)
data_sod = file_sod['raw_density']
exact_sod = file_sod['exact_density']
modes_sod = file_sod['raw_density_modes']
times_sod = file_sod['times']

filtered_sod, modes_sod_filtered, multiwavelets_siac_sod = apply_siac_filter(modes_sod, smoothness, modes_type='cfd', filter_size=siac_kernel_size, ghost_cells=10, N=N, pts_per_cell=pts_per_cell, xL=xL, xR=xR)
multiwavelets_siac_sod = compute_data_multiwavelets(modes_sod_filtered, modes_type='cfd', N=N)
disc_data_siac_sod, exact_disc_data_siac_sod = center_tc_data(filtered_sod, multiwavelets_siac_sod, exact_sod, eval_pts, half_window=half_window, pts_per_cell=pts_per_cell, threshold=threshold, debug=True)
disc_data_siac_sod_norm, exact_disc_data_siac_sod_norm, min_vals_sod, max_vals_sod, samples_norm_sod = normalize_clean_disc_data(disc_data_siac_sod, exact_disc_data_siac_sod, pts_per_cell=pts_per_cell, flat_val_threshold=flat_val_threshold, max_val_threshold=max_val_threshold)

# -- Validation data: 64 samples from T=1 to 1.5
file_sod_val = np.load('cfd_data/Sod/CFD_Sod_time1-1.5_N128_Samples64.npz', allow_pickle=True)
data_sod_val = file_sod_val['raw_density']
exact_sod_val = file_sod_val['exact_density']
modes_sod_val = file_sod_val['raw_density_modes']
times_sod_val = file_sod_val['times']

filtered_sod_val, modes_sod_val_filtered, multiwavelets_siac_sod_val = apply_siac_filter(modes_sod_val, smoothness, modes_type='cfd', filter_size=siac_kernel_size, ghost_cells=10, N=N, pts_per_cell=pts_per_cell, xL=xL, xR=xR)  
multiwavelets_siac_sod_val = compute_data_multiwavelets(modes_sod_val_filtered, modes_type='cfd', N=N)
disc_data_siac_sod_val, exact_disc_data_siac_sod_val = center_tc_data(filtered_sod_val, multiwavelets_siac_sod_val, exact_sod_val, eval_pts, half_window=half_window, pts_per_cell=pts_per_cell, threshold=threshold, debug=True)
disc_data_siac_sod_val_norm, exact_disc_data_siac_sod_val_norm, min_vals_sod_val, max_vals_sod_val, samples_norm_sod_val = normalize_clean_disc_data(disc_data_siac_sod_val, exact_disc_data_siac_sod_val, pts_per_cell=pts_per_cell, flat_val_threshold=flat_val_threshold, max_val_threshold=max_val_threshold)


# --- Sine-Entropy: 84 samples from T=1.5 to T=1.8
# Specific parameters for TCD threshold and flat val threshold needed for sine-entropy
half_window = 4
threshold = 0.9
flat_val_threshold= 1
max_val_threshold=0.1

file_sineEntropy = np.load('cfd_data/SineEntropy/CFD_SineEntropy_time1.5-1.8_N128_Samples84.npz', allow_pickle=True)
data_sineEntropy = file_sineEntropy['raw_density']
exact_sineEntropy = file_sineEntropy['exact_density']
modes_sineEntropy = file_sineEntropy['raw_density_modes']
times_sineEntropy = file_sineEntropy['times']

filtered_sineEntropy, modes_sineEntropy_filtered, multiwavelets_siac_sineEntropy = apply_siac_filter(modes_sineEntropy, smoothness, modes_type='cfd', filter_size=siac_kernel_size, ghost_cells=10, N=N, pts_per_cell=pts_per_cell, xL=xL, xR=xR)
multiwavelets_siac_sineEntropy = compute_data_multiwavelets(modes_sineEntropy_filtered, modes_type='cfd', N=N)
disc_data_siac_sineEntropy, exact_disc_data_siac_sineEntropy = center_tc_data(filtered_sineEntropy, multiwavelets_siac_sineEntropy, exact_sineEntropy, eval_pts, half_window=half_window, pts_per_cell=pts_per_cell, threshold=threshold, debug=True)
disc_data_siac_sineEntropy_norm, exact_disc_data_siac_sineEntropy_norm, min_vals_sineEntropy, max_vals_sineEntropy, samples_norm_sineEntropy = normalize_clean_disc_data(disc_data_siac_sineEntropy, exact_disc_data_siac_sineEntropy, pts_per_cell=pts_per_cell, flat_val_threshold=flat_val_threshold, max_val_threshold=max_val_threshold)


In [5]:
# Select TEST CFD discontinuity data

# --- Lax and Sod
cfd_label = 'LaxSod'
disc_data_siac_cfd = np.concatenate((disc_data_siac_lax, disc_data_siac_sod), axis=0)
exact_disc_data_siac_cfd = np.concatenate((exact_disc_data_siac_lax, exact_disc_data_siac_sod), axis=0)
disc_data_siac_cfd_norm = np.concatenate((disc_data_siac_lax_norm, disc_data_siac_sod_norm), axis=0)
exact_disc_data_siac_cfd_norm = np.concatenate((exact_disc_data_siac_lax_norm, exact_disc_data_siac_sod_norm), axis=0)
min_vals_test = np.concatenate((min_vals_lax, min_vals_sod), axis=0)
max_vals_test = np.concatenate((max_vals_lax, max_vals_sod), axis=0)
samples_norm_test = np.concatenate((samples_norm_lax, samples_norm_sod), axis=0)


In [6]:
# Select VALIDATION CFD discontinuity data

# --- Lax and Sod
cfd_label_val = 'LaxSod'
disc_data_siac_cfd_val = np.concatenate((disc_data_siac_lax_val, disc_data_siac_sod_val), axis=0)
exact_disc_data_siac_cfd_val = np.concatenate((exact_disc_data_siac_lax_val, exact_disc_data_siac_sod_val), axis=0)
disc_data_siac_cfd_val_norm = np.concatenate((disc_data_siac_lax_val_norm, disc_data_siac_sod_val_norm), axis=0)
exact_disc_data_siac_cfd_val_norm = np.concatenate((exact_disc_data_siac_lax_val_norm, exact_disc_data_siac_sod_val_norm), axis=0)
min_vals_val = np.concatenate((min_vals_lax_val, min_vals_sod_val), axis=0)
max_vals_val = np.concatenate((max_vals_lax_val, max_vals_sod_val), axis=0)
samples_norm_val = np.concatenate((samples_norm_lax_val, samples_norm_sod_val), axis=0)

In [ ]:
# Plot discontinuity data

num_samples_cfd = disc_data_siac_cfd_norm.shape[0]
shuffled_cfd = np.random.permutation(num_samples_cfd)
num_samples_cfd_val = disc_data_siac_cfd_val_norm.shape[0]
shuffled_cfd_val = np.random.permutation(num_samples_cfd_val)
num_samples = disc_data_siac_norm.shape[0]
shuffled = np.random.permutation(num_samples)
plt_samples = 20
xvals = np.linspace(-half_window, half_window+1, (2*half_window+1)*pts_per_cell)

# Plot CFD Test discontinuity data
plt.subplots(1, 3, figsize=(16, 4))
plt.subplot(1, 3, 1)
for sample in shuffled_cfd[:plt_samples]:
    plt.plot(xvals, disc_data_siac_cfd_norm[sample, :])
plt.title('Norm. CFD Test disc. data: '+str(plt_samples))

# Plot CFD Validation discontinuity data
plt.subplot(1, 3, 2)
for sample in shuffled_cfd_val[:plt_samples]:
    plt.plot(xvals, disc_data_siac_cfd_val_norm[sample, :])
plt.title('CFD Validation disc. data: '+str(plt_samples))

# Plot Synthetic discontinuity data
plt.subplot(1, 3, 3)
for sample in shuffled[:plt_samples]:
    plt.plot(xvals, disc_data_siac_norm[sample, :])
plt.title('Synthetic Train disc. data: '+str(plt_samples))

label_ic_params = 'Tophat'
label_cfd_extra = '_LaxSodVal'



In [8]:
# Save the data  - 

# Synthetic Training Data
num_train = disc_data_siac_norm.shape[0]
shuffled_indices = np.random.permutation(num_train)
disc_data_train = disc_data_siac_norm[shuffled_indices, :].reshape((num_train, 1, -1))
exact_disc_data_train = exact_disc_data_siac_norm[shuffled_indices, :].reshape((num_train, 1, -1))
min_vals_train = min_vals_synthetic[shuffled_indices]
max_vals_train = max_vals_synthetic[shuffled_indices]

# CFD Validation Data
num_val = disc_data_siac_cfd_val_norm.shape[0]
max_samples_val = 50
shuffled_indices_val = np.random.permutation(num_val)[:max_samples_val]
disc_data_val = disc_data_siac_cfd_val_norm[shuffled_indices_val, :].reshape((max_samples_val, 1, -1))
exact_disc_data_val = exact_disc_data_siac_cfd_val_norm[shuffled_indices_val, :].reshape((max_samples_val, 1, -1))
min_vals_val = min_vals_val[shuffled_indices_val]
max_vals_val = max_vals_val[shuffled_indices_val]

# CFD Test Data
num_test = disc_data_siac_cfd_norm.shape[0]
shuffled_indices_test = np.random.permutation(num_test)
disc_data_test = disc_data_siac_cfd_norm[shuffled_indices_test, :].reshape((num_test, 1, -1))
exact_disc_data_test = exact_disc_data_siac_cfd_norm[shuffled_indices_test, :].reshape((num_test, 1, -1))
min_vals_test = min_vals_test[shuffled_indices_test]
max_vals_test = max_vals_test[shuffled_indices_test]



np.savez(training_label+ 'disc_data_norm_siac'+siac_kernel_size+'_syntheticTrain'+label_ic_params+str(disc_data_siac_norm.shape[0])+
         '_test'+cfd_label+str(num_test)+
         '_val'+cfd_label_val+ str(max_samples_val)+
         label_cfd_extra+'.npz', disc_train=disc_data_train, exact_train=exact_disc_data_train, 
         disc_val=disc_data_val, exact_val=exact_disc_data_val,
         disc_test=disc_data_test, exact_test=exact_disc_data_test, 
         min_train=min_vals_train, max_train=max_vals_train, 
            min_val=min_vals_val, max_val=max_vals_val,
         min_test=min_vals_test, max_test=max_vals_test)